# 2357. Make Array Zero by Subtracting Equal Amounts

[Leetcode 2357 Make Array Zero by Subtracting Equal Amounts](https://leetcode.com/problems/make-array-zero-by-subtracting-equal-amounts/description/) is a problem on the [Amazon Questions List](https://leetcode.com/problem-list/7p5x763/?sorting=W3sic29ydE9yZGVyIjoiREVTQ0VORElORyIsIm9yZGVyQnkiOiJGUkVRVUVOQ1kifV0%3D&page=1)

## In this notebook

We're going to go over all the logic necessary to solve the problem as fast as possible, as well as (some of) the observations required to get there.

> You are given a non-negative integer array nums. In one operation, you must:
>
>    - Choose a positive integer x such that x is less than or equal to the smallest non-zero element in nums.
>    - Subtract x from every positive element in nums.
>Return the minimum number of operations to make every element in nums equal to 0.
>
> Constraints:
>
>    - 1 <= nums.length <= 100
>    - 0 <= nums[i] <= 100
>

There's a couple interesting stipulations here.
First, our input list is small. Less than 100 numbers is so small we could easily justify iterating over the list naively.

Secondly, no number in the list will exceed 100. This matters less for us because we're using Python, but that would mean that certain languages could get away with 8 bit numbers.

> Example 1:
> 
>    - Input: nums = [1,5,0,3,5]
>    - Output: 3
>
> 
> Example 2:
> 
>    - Input: nums = [0]
>    - Output: 0

Let's start by making some obervations about these rules and examples.

*Observation 1*: An operation is defined by the problem as both choosing the number `x` and subtracting that number from all positive elements in nums. We then have done 1 operation and would count up.

In psuedo code, that might look something like this:


In [15]:
def operation(nums):
    """
    Find x (which is less than or equal to the smallest non-zero element in nums)
    Subtract x from all nums
    """

def make_array_zero(nums):
    """
    let operations = 0
    while any non-zero num in nums:
        do operation(nums)
        operations += 1
    return operations
    """

Before writing any code, let's consider how we could find `x`. 

Let's inspect that first rule again:
> Choose a positive integer x such that x is less than or equal to the smallest non-zero element in nums.
 
### Question: Does it make sense to ever pick an element smaller than the smallest non-zero element?
Walking through Example 1:
```python
nums = [1,5,0,3,5]
# For our first choice, we must pick 1. (1 <= 1 && 1 != 0)
# After we complete operation(nums), we now have
nums = [0, 4, 0, 2, 4]
#What if we choose 1 again? Then we would have
# nums = [0, 3, 0, 1, 4]
# ...Forcing us to choose 1 again. If we had picked 2, we would have saved an operation in the final output.
```
### Conclusion:
*There is no reason to ever pick an element smaller than the smallest element.*

We should pick the smallest element, whatever it is. Let's code that up, remembering that the size of nums is supposed to be small.

In [16]:
def operation(nums):
    def get_smallest_non_zero(nums):
        # constraint from our input size. Doesn't really matter what it is as long as it's at least as big as that.
        smallest = 100 
        
        for n in nums:
            if n > 0 and n <= smallest:
                smallest = n
        return smallest
    
    assert nums

    x = get_smallest_non_zero(nums) # We can't use min since it will return 0. Hmm.
    operated_list = []
    
    for n in nums:
        value = 0
        if x <= n:
            value = n - x
        operated_list.append(value)
    return operated_list

def make_array_zero(nums):
    operations = 0

    while any(nums):
        nums = operation(nums)
        operations += 1
    return operations

tests = [
    [1,5,0,3,5],
    [0],
    [0,1,2,3,4,5,6,7,8,9,],
]

for test in tests:
    print(f"{test}")
    print(make_array_zero(test[:]))


[1, 5, 0, 3, 5]
3
[0]
0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
9


### What next?

The real question of the problem is "What is the best way to find the minimum element, so that we can subtract it from the other elements?"

At this point, there are several reasonable ways to go about finding `x` and solving the problem. 

- Sort the input (remember, the length of nums is guaranteed to be small)
- Use some sort of data structure, like a heap, to maintain an invariant for us so it's easier to count operations and solve the problem.

We're going to go with option 2 for this notebook. 

There's a minor hiccup here, which is that 0 is the smallest number in the heap. We'll have to do something about that. What if we just... removed them?

(That's forshadowing for later.)

In [17]:
import heapq # https://docs.python.org/3/library/heapq.html

def operation(nums):
    assert nums

    while nums[0] == 0:
        heapq.heappop(nums)
    x = heapq.heappop(nums)

    operated_list = []
    assert 0 not in nums
    for n in nums:
        assert x <= n, f"x: {x}, n: {n}"

        value = 0
        if x <= n:
            value = n - x    
        operated_list.append(value)
    return operated_list

def make_array_zero(nums):
    operations = 0
    heapq.heapify(nums)
    while any(nums):
        nums = operation(nums)
        operations += 1
    return operations

tests = [
    [1,5,0,3,5],
    [0],
    [0,1,2,3,4,5,6,7,8,9,],
]

for test in tests:
    print(f"{test}")
    print(make_array_zero(test[:]))


[1, 5, 0, 3, 5]
3
[0]
0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
9


### Those Pesky Zeros

This *works*, or at least the idea does, but it's kinda weird that we're forced to remove the 0s like that every time.

Actually, why add on the new 0's we created at all? Do they matter? The only thing we care about is the number of times we call the `operation` function, right?

(Right)

So we don't care about the 0's *at all*. We can condense our previous code.

In [18]:
def operation(nums):
    assert nums
    x = heapq.heappop(nums)
    operated_list = []
    for n in nums:
        assert x <= n, f"x: {x}, n: {n}"
        if n - x > 0:
            heapq.heappush(operated_list, (n - x))
    return operated_list

def make_array_zero(nums):
    operations = 0
    nums = [n for n in nums if n > 0]
    heapq.heapify(nums)
    while nums:
        operations += 1
        nums = operation(nums)
    return operations

for test in tests:
    print(f"{test}")
    print(make_array_zero(test[:]))

[1, 5, 0, 3, 5]
3
[0]
0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
9


But, hold on. Did you notice something odd about the third example? Actually, that first one too!

The output is actually the number of non-zero elements in the list. In fact, that's true for all of them. It's just more obvious because that one is sorted.

*Observation 2*: We solved it above by knowing the smallest number in the list and decrementing every other number. That means we had to know every number at some point. We've actually known this since our brute force version!

*Observation 3*: We could change `operation(nums)` to instead just remove all instances of `x` and return the list and it would be the same result.

In [19]:
def operation(nums):
    assert nums
    x = heapq.heappop(nums)
    operated_list = []
    for n in nums:
        if x != n:
            heapq.heappush(operated_list, n)
    return operated_list

for test in tests:
    print(f"{test}")
    print(make_array_zero(test[:]))

[1, 5, 0, 3, 5]
3
[0]
0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
9


*Observation 4*: Not counting duplicates is strikingly similar to how sets work.

*Observation 5*: Since we don't care about either 0's OR duplicates, the minimum number of ops is the number of unique non-zero elements in the list.

We can now further improve our logic to something like this. We now no longer care about the ordering, because we're switching to a set.

In [20]:
def make_array_zero(nums):
    operations = set()
    for n in nums:
        if n > 0:
            operations.add(n)
    return len(operations)

for test in tests:
    print(f"{test}")
    print(make_array_zero(test[:]))


[1, 5, 0, 3, 5]
3
[0]
0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
9


Finally, we can optimize. An optimal solution looks something like the cell below.

There's no longer an explicit call to our `operation()`, it's now implicit in the call.

In [21]:
def make_array_zero(nums):
    unique_nums = set(nums)
    return len(unique_nums) - 1 if 0 in unique_nums else len(unique_nums)

tests = [
    [1,5,0,3,5],
    [0],
    [0,1,2,3,4,5,6,7,8,9,],
]

for test in tests:
    print(f"{test}")
    print(make_array_zero(test[:]))

[1, 5, 0, 3, 5]
3
[0]
0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
9


### On Optimization
There's an extremely subtle optimization (that doesn't matter) in our last code block. It is faster to indiscriminately add everything to the set and then check the set members for 0 than it would be to check every number going in.

### Playground
While the original problem calls for only 100 numbers with all being no more than 100, our code will still work for much bigger numbers.

In [22]:
def output_tests():
    for test in tests:
        # These 10s here are arbitrary. You can put in whatever you like.
        test_str = f"{test}" if len(test) <= 10 else f"{test[:10]}..."
        print(test_str)
        print(make_array_zero(test[:]))

tests = [
    [1,5,0,3,5],
    [0],
    [0,1,2,3,4,5,6,7,8,9,],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
    [i for i in range(0, 100, 2)],
    [i for i in range(1, 1_000_000, 1)],
]

output_tests()
    

[1, 5, 0, 3, 5]
3
[0]
0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
9
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...
0
[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]...
49
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]...
999999


### Alternate - Sorting Method

While not strictly necessary to understand that this was actually set math masquerading as a coding problem, here is code that will work if you sort the input. This version also works around the problem of 0 being the min by removing it, but you could use `get_smallest_non_zero(nums)` for this as well.

In [24]:
def operation(nums):
    assert nums
    x = nums[0]
    assert min(nums) == x
    return [n - x for n in nums if n - x > 0]

def make_array_zero(nums):
    operations = 0
    nums.sort()
    nums = [n for n in nums if n > 0]
    while nums:
        operations += 1
        nums = operation(nums)
    return operations

tests = [
    [1,5,0,3,5],
    [0],
    [0,1,2,3,4,5,6,7,8,9,],
]

for test in tests:
    print(f"{test}")
    print(make_array_zero(test[:]))

[1, 5, 0, 3, 5]
3
[0]
0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
9


A couple things to note about this solution:

- If we change the list to not be sorted by no longer calling `nums.sort()`, we can simply change the list comprehension in operation to `return [n for n in nums if n != x]`
- Further optimizations will take us down set math which should lead to the one-line solution above.

# Happy coding!